In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers
!pip install datasets
!pip install wandb
!pip install -U datasets
!pip install objaverse
!pip install diffusers
!pip install trimesh
!pip install jaxtyping
!pip install pytorch-lightning
!pip install ijson
!pip install triton==3.2.0
!pip install wandb
!pip uninstall -y pillow
!pip install pillow==9.5.0 --no-cache-dir

In [ ]:
!git clone --depth 1 --branch main https://github.com/stepfun-ai/Step1X-3D.git

In [ ]:
!cp "/kaggle/working/AI-Game-Engine/Step1x3d repo script patched/mesh_render.py" "/kaggle/working/Step1X-3D/step1x3d_texture/differentiable_renderer/mesh_render.py"
!ls -l "/kaggle/working/Step1X-3D/step1x3d_texture/differentiable_renderer/mesh_render.py"
print("\n✅ File replaced successfully.")

In [1]:
import sys
sys.path.append("./Step1X-3D")
!pip install -r ./Step1X-3D/requirements.txt --verbose

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
  Cloning https://github.com/NVlabs/nvdiffrast.git to /tmp/pip-req-build-7grtduux
  Running command git version
  git version 2.34.1
  Running command git clone --filter=blob:none https://github.com/NVlabs/nvdiffrast.git /tmp/pip-req-build-7grtduux
  Cloning into '/tmp/pip-req-build-7grtduux'...
  Running command git rev-parse HEAD
  729261dc64c4241ea36efda84fbf532cc8b425b8
  Resolved https://github.com/NVlabs/nvdiffrast.git to commit 729261dc64c4241ea36efda84fbf532cc8b425b8
  Running command git rev-parse HEAD
  729261dc64c4241ea36efda84fbf532cc8b425b8
  Preparing metadata (setup.py) ...   Running command python setup.py egg_info
done
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-11_t9byr
  Running command git version
  git version 2.34.1
  Running command git clone --filter=blob:none https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-bui

In [ ]:
!pip install torch-cluster -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!apt-get update && apt-get install -y libaio-dev

In [ ]:
import sys
for name in list(sys.modules):
    if name.startswith("mmfreelm"):
        del sys.modules[name]
print("Removed mmfreelm from sys.modules.")
sys.path = [p for p in sys.path if "/kaggle/working/AI-Game-Engine" not in p]
print("Cleaned sys.path entry for AI-Game-Engine.")

In [2]:
!rm -rf AI-Game-Engine
!git clone https://github.com/Sakib323/AI-Game-Engine.git
sys.path.append('/kaggle/working/AI-Game-Engine')
from mmfreelm.models.hgrn_bit.mesh_dit import MeshDiT_models

Cloning into 'AI-Game-Engine'...
remote: Enumerating objects: 1088, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 1088 (delta 62), reused 61 (delta 32), pack-reused 995 (from 1)
Receiving objects: 100% (1088/1088), 463.05 MiB | 50.07 MiB/s, done.
Resolving deltas: 100% (709/709), done.
Updating files: 100% (228/228), done.


2025-10-11 13:52:22.824134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760190743.000481    2055 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760190743.050549    2055 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# =========================================================================================
# TRAIN MESH GENERATION MODEL: DOWNLOAD DATASET & TRAIN THE MODEL
# This script download the dataset then process them and lastly train the model.
# =========================================================================================

In [5]:
import requests
import gzip
import json
import objaverse
import gc
import os
import ijson
import json
import objaverse
import pathlib
import shutil
import os
import os
import json
import torch
import torch.nn.functional as F
import traceback
import numpy as np
import trimesh
from tqdm import tqdm
import logging
import random
from torch.utils.data import Dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments
from safetensors.torch import load_file as safetensors_load
import wandb
import shutil
from diffusers import DDIMScheduler
from mmfreelm.models.hgrn_bit.mesh_dit import MeshDiT_models
from step1x3d_geometry.models.pipelines.pipeline import Step1X3DGeometryPipeline
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32
print(f"Using device: {device}, dtype: {dtype}")

Using device: cuda, dtype: torch.float32


In [ ]:
KEYS_TO_EXCLUDE = {
    "tags", "name", "staffpickedAt", "viewCount", "likeCount", "animationCount",
    "commentCount", "publishedAt", "user", "description", "faceCount", "createdAt",
    "vertexCount", "license", "uri", "viewerUrl", "embedUrl", "isDownloadable",
    "categories", "isAgeRestricted", "archives"
}

print("Starting dataset preparation...")

url = "https://huggingface.co/datasets/tiange/Cap3D/resolve/main/Objaverse_files/cap3d_captions.json.gz"
captions_file = 'cap3d_captions.json.gz'
if not os.path.exists(captions_file):
    print("Downloading captions file...")
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(captions_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Captions file downloaded successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading captions: {e}")
        exit()
else:
    print("Captions file already exists, skipping download.")

def process_batch(batch, f, total_counter):
    uids = [uid for uid, _ in batch]
    try:
        annotations = objaverse.load_annotations(uids)
    except Exception as e:
        print(f"Error loading annotations for batch: {e}")
        return 0  # Skip this batch on error

    for i, (uid, caption) in enumerate(batch):
        metadata = annotations.get(uid, {})
        filtered_metadata = {k: v for k, v in metadata.items() if k not in KEYS_TO_EXCLUDE}

        # Filter thumbnails to keep only 1024x576 images
        if "thumbnails" in filtered_metadata and "images" in filtered_metadata["thumbnails"]:
            filtered_metadata["thumbnails"]["images"] = [
                img for img in filtered_metadata["thumbnails"]["images"]
                if img.get("width") == 1024 and img.get("height") == 576
            ]

        datapoint = {
            "uid": uid,
            "caption": caption,
            **filtered_metadata
        }
        f.write(json.dumps(datapoint) + '\n')
        if total_counter + i < 10:
            print(f"Datapoint {uid}:")
            print(json.dumps(datapoint, indent=2))
            print("-" * 50)
    del annotations
    gc.collect()
    return len(batch)


batch_size = 100000
total_counter = 0
with open('extended_dataset.jsonl', 'w', encoding='utf-8') as f:
    batch = []
    for uid, caption in ijson.kvitems(gzip.open(captions_file, 'rt', encoding='utf-8'), ''):
        batch.append((uid, caption))
        if len(batch) >= batch_size:
            processed = process_batch(batch, f, total_counter)
            total_counter += processed
            print(f"Processed {total_counter} captions so far.")
            batch = []
    if batch:
        processed = process_batch(batch, f, total_counter)
        total_counter += processed

print(f"Total entries in extended dataset: {total_counter}")
print("Dataset saved to 'extended_dataset.jsonl'")
print("Script finished.")

In [ ]:
jsonl_path = "extended_dataset.jsonl"
output_dir = pathlib.Path("downloads")
output_dir.mkdir(exist_ok=True)
metadata_output_path = output_dir / "metadata.json"
download_limit = 1500

uid_metadata = {}
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        if len(uid_metadata) >= download_limit:
            break
        entry = json.loads(line)
        uid = entry["uid"]
        caption = entry.get("caption", "")
        thumbnails = entry.get("thumbnails", {}).get("images", [])
        thumbnail_urls = [img["url"] for img in thumbnails if "url" in img]

        uid_metadata[uid] = {
            "caption": caption,
            "thumbnails": thumbnail_urls
        }

uids = list(uid_metadata.keys())
print(f"Downloading {len(uids)} models...")
paths = objaverse.load_objects(uids=uids)

final_metadata = {}
for uid, src_path in paths.items():
    dst_path = output_dir / f"{uid}.glb"
    try:
        shutil.copy(src_path, dst_path)
        final_metadata[uid] = uid_metadata[uid]
        print(f"Saved {uid} to {dst_path}")
    except Exception as e:
        print(f"Failed to save {uid}: {e}")

with open(metadata_output_path, "w", encoding="utf-8") as f:
    json.dump(final_metadata, f, indent=2)

print(f"\nDownloaded {len(final_metadata)} models.")
print(f"Metadata saved to {metadata_output_path}")


In [ ]:
print("Initializing Step1X-3D VAE...")
geometry_pipeline = Step1X3DGeometryPipeline.from_pretrained("stepfun-ai/Step1X-3D", subfolder='Step1X-3D-Geometry-1300m',torch_dtype=dtype)
vae = geometry_pipeline.vae.to(device)
vae.eval()
print("Step1X-3D VAE initialized successfully.")

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer loaded.")

In [ ]:
def load_local_dataset(dataset_dir):
    downloaddir="downloads"
    metadata_file="downloads/metadata.json"
    if not os.path.exists(metadata_file):
        print(f"Metadata file not found at: {metadata_file}")
        return [], []
        
    with open(metadata_file, 'r', encoding='utf-8') as f:
        metadata = json.load(f)

    glb_paths, texts = [], []
    for uid, info in tqdm(metadata.items(), desc="Loading GLB files"):
        glb_path = os.path.join(downloaddir, f"{uid}.glb")
        caption = info.get('caption', '').strip()

        if not os.path.isfile(glb_path):
            continue

        glb_paths.append(glb_path)
        texts.append(caption)

    print(f"Loaded {len(glb_paths)} GLB files with captions.")
    return glb_paths, texts

In [ ]:
def sample_uniform_and_sharp_points(mesh, num_uniform_points=16384, num_sharp_points=16384, sharp_threshold_deg=60):
    uniform_points, face_indices_uniform = trimesh.sample.sample_surface(mesh, num_uniform_points)
    uniform_normals = mesh.face_normals[face_indices_uniform]
    sharp_points, sharp_normals = None, None

    try:
        edge_angles = mesh.face_adjacency_angles
        sharp_threshold_rad = np.deg2rad(sharp_threshold_deg)
        sharp_edge_indices = np.where(edge_angles > sharp_threshold_rad)[0]

        if len(sharp_edge_indices) > 0:
            face_indices_of_sharp_edges = mesh.face_adjacency[sharp_edge_indices].flatten()
            sharp_face_indices = np.unique(face_indices_of_sharp_edges)
            if len(sharp_face_indices) > 0:
                sharp_mesh = mesh.submesh([sharp_face_indices], append=True)
                if sharp_mesh.vertices.shape[0] > 3 and sharp_mesh.faces.shape[0] > 1:
                    sharp_points, face_indices_sharp = trimesh.sample.sample_surface(sharp_mesh, num_sharp_points)
                    sharp_normals = sharp_mesh.face_normals[face_indices_sharp]
    except Exception as e:
        logger.warning(f"Could not perform sharp sampling due to a mesh error: {e}. Falling back to uniform.")
        sharp_points = None

    if sharp_points is None or sharp_normals is None:
        logger.info("No sharp regions found or submesh failed. Using uniform samples for sharp set.")
        sharp_points, face_indices_sharp = trimesh.sample.sample_surface(mesh, num_sharp_points)
        sharp_normals = mesh.face_normals[face_indices_sharp]

    return (uniform_points, uniform_normals), (sharp_points, sharp_normals)

In [ ]:
def process_mesh_to_vae_input(mesh_path, num_points=32768):
    try:
        mesh = trimesh.load(mesh_path, force='mesh', process=True)
        if isinstance(mesh, trimesh.Scene):
            if not mesh.geometry:
                logger.warning(f"Skipping {mesh_path}: Trimesh scene is empty.")
                return None
            mesh = mesh.dump().sum()

        if not isinstance(mesh, trimesh.Trimesh) or len(mesh.vertices) == 0 or len(mesh.faces) == 0:
            logger.warning(f"Skipping {mesh_path}: No valid mesh data.")
            return None

        if not mesh.is_watertight:
            trimesh.repair.fill_holes(mesh)

        center = mesh.bounds.mean(axis=0)
        mesh.apply_translation(-center)
        max_extent = np.max(np.linalg.norm(mesh.vertices, axis=1))
        if max_extent > 1e-6:
            mesh.apply_scale(1.0 / max_extent)

        (uniform_points, uniform_normals), (sharp_points, sharp_normals) = sample_uniform_and_sharp_points(
            mesh, num_uniform_points=num_points, num_sharp_points=num_points
        )

        surface_cloud = np.hstack([uniform_points, uniform_normals])
        sharp_cloud = np.hstack([sharp_points, sharp_normals])

        return {
            "surface": torch.tensor(surface_cloud, dtype=dtype).unsqueeze(0),
            "sharp_surface": torch.tensor(sharp_cloud, dtype=dtype).unsqueeze(0)
        }
    except Exception as e:
        logger.error(f"CRITICAL ERROR processing {mesh_path}: {str(e)}\n{traceback.format_exc()}")
        return None

In [ ]:
def create_dataset_from_local_files(glb_paths, texts):
    logger.info("Starting dataset creation from local files...")
    processed_data = []

    for mesh_path, caption in tqdm(zip(glb_paths, texts), total=len(glb_paths), desc="Processing Local Meshes"):
        mesh_inputs = process_mesh_to_vae_input(mesh_path, num_points=16384)
        if mesh_inputs is None:
            continue

        mesh_inputs_on_device = {k: v.to(device) for k, v in mesh_inputs.items()}

        with torch.no_grad():
            # Encode mesh to get 3D latent vector
            _shape_embeds, kl_embed, _posterior = vae.encode(sample_posterior=True, **mesh_inputs_on_device)
            latent_3d = kl_embed.squeeze(0).cpu()

        # Tokenize text
        tokens = tokenizer(caption, padding="max_length", max_length=128, truncation=True, return_tensors="pt")

        processed_data.append({
            "x": latent_3d,
            "y": {
                "input_ids": tokens["input_ids"].squeeze(0),
                "attention_mask": tokens["attention_mask"].squeeze(0),
            }
        })

    logger.info(f"Successfully created dataset with {len(processed_data)} samples.")
    if not processed_data:
        logger.critical("Dataset is empty. Check logs for processing errors.")

    return processed_data

In [ ]:
class MeshDataset(Dataset):
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

class CustomDataCollator:
    def __call__(self, features):
        batch = {}
        batch['x'] = torch.stack([f['x'] for f in features])
        y_features = [f['y'] for f in features]
        batch['y'] = {key: torch.stack([d[key] for d in y_features]) for key in y_features[0]}
        return batch

class MeshDiTTrainer(Trainer):
    def __init__(self, *args, noise_scheduler, **kwargs):
        super().__init__(*args, **kwargs)
        self.noise_scheduler = noise_scheduler

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        x_start = inputs.get("x")
        model_kwargs = inputs.get("y")  # The model's forward pass expects 'y'
        noise = torch.randn_like(x_start)        
        timesteps = torch.randint(0, self.noise_scheduler.config.num_train_timesteps, (x_start.shape[0],), device=x_start.device).long()        
        noisy_latents = self.noise_scheduler.add_noise(x_start, noise, timesteps)        
        noise_pred = model(noisy_latents, timesteps, model_kwargs)
        loss = F.mse_loss(noise_pred, noise)
        
        return (loss, {"loss": loss}) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        inputs = self._prepare_inputs(inputs)
        with torch.no_grad():
            loss = self.compute_loss(model, inputs)
        return (loss.detach(), None, None)


# =========================================================================================
# PHASE 1 OF MESH GENERATION MODEL TRAINING
# Smaller latent set size (2048), a higher learning rate (1e-4), and a higher batch size (2)
# =========================================================================================

In [ ]:
def train_model():
    WANDB_TOKEN = "89b06c10468af620747b4bd340f72fa5d56f6849"
    try:
        wandb.login(key=WANDB_TOKEN)
        os.environ["WANDB_PROJECT"] = "mesh-dit-3d-generation"  # your preferred project name
        use_wandb = True
        print("✅ Successfully logged in to W&B.")
    except Exception as e:
        print(f"⚠️ Could not log in to W&B: {e}. Disabling W&B logging.")
        use_wandb = False

    DATASET_PATH = "AI-Game-Engine/testing_dataset"
    if not os.path.isdir(DATASET_PATH):
        print("=" * 80)
        print(f"FATAL ERROR: Dataset directory not found at: '{DATASET_PATH}'")
        print("Please ensure the dataset path is correct.")
        print("=" * 80)
        return

    glb_paths, texts = load_local_dataset(dataset_dir=DATASET_PATH)
    all_data = create_dataset_from_local_files(glb_paths, texts)

    print("\n🔥 Cleaning up .glb files to free space...")
    shutil.rmtree("downloads", ignore_errors=True)
    gc.collect()
    print("✅ All GLB files removed.")


    if not all_data:
        print("FATAL: Dataset creation failed. No data to train on. Exiting.")
        return

    random.shuffle(all_data)
    eval_size = max(1, int(len(all_data) * 0.10))
    train_data = all_data[eval_size:]
    eval_data = all_data[:eval_size]

    print(f"Data split: {len(train_data)} training samples, {len(eval_data)} evaluation samples.")

    train_dataset = MeshDataset(train_data)
    eval_dataset = MeshDataset(eval_data)
    data_collator = CustomDataCollator()

    noise_scheduler = DDIMScheduler(
        num_train_timesteps=1000,
        beta_schedule="linear",
        prediction_type="epsilon",
        clip_sample=False,
    )

    # =================================================================
    # PHASE 1 TRAINING
    # =================================================================
    print("\n" + "=" * 50)
    print(" Starting MeshDiT Training: PHASE 1")
    print("=" * 50 + "\n")

    model_p1 = MeshDiT_models['MeshDiT-S'](
        input_tokens=2048,
        vocab_size=tokenizer.vocab_size,
        use_rope=False,
        use_ternary_rope=False,
        image_condition=False,
        full_precision=False,
        optimized_bitlinear=False,
    ).to(device, dtype=dtype)

    training_args_p1 = TrainingArguments(
        output_dir="./mesh_dit_phase1_checkpoint",
        num_train_epochs=25,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=1e-4,
        lr_scheduler_type="cosine",
        weight_decay=0.01,
        warmup_steps=200,
        logging_dir='./logs_phase1',
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True if device.type == "cuda" else False,
        max_grad_norm=1.0,
        dataloader_num_workers=2,
        remove_unused_columns=False,
        # --- Step 3: Set the report_to argument ---
        report_to="wandb" if use_wandb else "tensorboard",
        run_name="MeshDiT-S-Phase1-FullPrecision",
    )

    trainer_p1 = MeshDiTTrainer(
        model=model_p1,
        args=training_args_p1,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        noise_scheduler=noise_scheduler,    )

    trainer_p1.train()
    trainer_p1.save_model("./mesh_dit_phase1_final")
    tokenizer.save_pretrained("./mesh_dit_phase1_final")
    print("Phase 1 training complete. Model saved to ./mesh_dit_phase1_final")

    print("\nCleaning up old checkpoints to free up disk space...")
    shutil.rmtree('./mesh_dit_phase1_checkpoint', ignore_errors=True)
    shutil.rmtree('./logs_phase1', ignore_errors=True)
    print("Cleanup complete.")

    if use_wandb:
        wandb.finish()  # End the first W&B run

    # =================================================================
    # PHASE 2 TRAINING (Fine-tuning)
    # =================================================================
    print("\n" + "=" * 50)
    print(" Starting MeshDiT Training: PHASE 2 (Fine-tuning)")
    print("=" * 50 + "\n")

    model_p2 = MeshDiT_models['MeshDiT-S'](
        input_tokens=2048,
        vocab_size=tokenizer.vocab_size,
        use_rope=False,
        use_ternary_rope=False,
        image_condition=False,
        full_precision=False,
        optimized_bitlinear=False,
    ).to(device, dtype=dtype)

    print("Loading weights from Phase 1 model for fine-tuning...")
    state_dict_p1 = safetensors_load("./mesh_dit_phase1_final/model.safetensors", device="cpu")
    model_p2.load_state_dict(state_dict_p1)
    print("Weights loaded successfully.")

    training_args_p2 = TrainingArguments(
        output_dir="./mesh_dit_checkpoint",
        num_train_epochs=25,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=16,
        learning_rate=5e-5,
        lr_scheduler_type="cosine",
        weight_decay=0.01,
        warmup_steps=200,
        logging_dir='./logs_phase2',
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True if device.type == "cuda" else False,
        max_grad_norm=1.0,
        dataloader_num_workers=2,
        remove_unused_columns=False,
        # --- Step 3 (Again): Set the report_to argument ---
        report_to="wandb" if use_wandb else "tensorboard",
        run_name="MeshDiT-S-Phase2-FineTuning",
    )

    trainer_p2 = MeshDiTTrainer(
        model=model_p2,
        args=training_args_p2,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        noise_scheduler=noise_scheduler,    )

    trainer_p2.train()

    print("\nPhase 2 training complete. Saving the final model.")
    trainer_p2.save_model("./mesh_dit_final")
    tokenizer.save_pretrained("./mesh_dit_final")
    print("Final model saved to ./mesh_dit_final")

    if use_wandb:
        wandb.finish()


In [ ]:
if __name__ == "__main__":
    train_model()

In [ ]:
import os
import torch
import numpy as np
import trimesh
from tqdm import tqdm
from transformers import AutoTokenizer
from safetensors.torch import load_file as safetensors_load
from diffusers import DDIMScheduler
from step1x3d_geometry.models.pipelines.pipeline import Step1X3DGeometryPipeline
MODEL_PATH = "./mesh_dit_final"
OUTPUT_FILENAME = "generated_object.glb"
TEXT_PROMPT = "Sofa"
CFG_SCALE_TEXT = 7.5
NUM_SAMPLING_STEPS = 50 

INPUT_TOKENS = 2048 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

print("\n" + "="*50)
print("      Starting Mesh Generation")
print("="*50 + "\n")

if not os.path.exists(MODEL_PATH):
    print(f"Model path not found: {MODEL_PATH}. Please ensure the trained model is available.")
    exit()

# --- Load Models ---
print(f"Loading models from {MODEL_PATH} onto {device}...")

# 1. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
generation_model = MeshDiT_models['MeshDiT-S'](
    input_tokens=INPUT_TOKENS,
    vocab_size=tokenizer.vocab_size,
    use_rope=False,
    use_ternary_rope=False,
    image_condition=False,
    full_precision=False,
    optimized_bitlinear=False, # Must match training config
).to(device, dtype=dtype).eval()

state_dict = safetensors_load(os.path.join(MODEL_PATH, "model.safetensors"), device=str(device))
generation_model.load_state_dict(state_dict)
print("-> DiT model loaded.")

# 3. Load the 3D VAE
try:
    geometry_pipeline = Step1X3DGeometryPipeline.from_pretrained(
        "stepfun-ai/Step1X-3D",
        subfolder='Step1X-3D-Geometry-1300m',
        torch_dtype=dtype
    )
    vae = geometry_pipeline.vae.to(device).eval()
    print("-> 3D VAE loaded.")
except Exception as e:
    print(f"Error loading VAE pipeline: {e}")
    exit()

# 4. Initialize the DDIM Noise Scheduler
scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_schedule="linear",
    prediction_type="epsilon",
    clip_sample=False,
)
scheduler.set_timesteps(NUM_SAMPLING_STEPS)
print("-> DDIM scheduler ready.")

# --- Prepare Generation Inputs for Classifier-Free Guidance ---
print("--- Preparing Generation Inputs ---")
tokens = tokenizer(TEXT_PROMPT, padding="max_length", max_length=128, truncation=True, return_tensors="pt")

# Conditional input (the actual text prompt)
cond_input_ids = tokens["input_ids"].to(device)
cond_attention_mask = tokens["attention_mask"].to(device)

# Unconditional input (a null prompt) for guidance
uncond_input_ids = torch.zeros_like(cond_input_ids)
uncond_attention_mask = torch.zeros_like(cond_attention_mask)

# Combine for a single batch
y_in = {
    "input_ids": torch.cat([cond_input_ids, uncond_input_ids], dim=0),
    "attention_mask": torch.cat([cond_attention_mask, uncond_attention_mask], dim=0)
}
# Add a dummy image latent tensor as the model expects it, even if not used
y_in["image_latent"] = torch.zeros(
    (y_in["input_ids"].shape[0], 4, 64, 64), 
    device=device, 
    dtype=dtype
)
print(f"-> Text prompt tokenized: '{TEXT_PROMPT}'")

# --- Denoising Process using the DDIM Scheduler ---
print("--- Starting Denoising Process ---")
# Start with random noise
latent = torch.randn(1, INPUT_TOKENS, 64, device=device, dtype=dtype)

with torch.no_grad():
    for t in tqdm(scheduler.timesteps, desc="DDIM Sampling"):
        # Expand the latents for Classifier-Free Guidance
        latent_model_input = torch.cat([latent] * 2)
        
        # The model's forward_with_cfg handles the two forward passes and CFG scaling internally
        # FIXED: Move the timestep tensor `t` to the correct device
        noise_pred = generation_model.forward_with_cfg(
            x=latent_model_input,
            t=t.to(device).repeat(2),
            y=y_in,
            cfg_scale_text=CFG_SCALE_TEXT,
            cfg_scale_image=0 # Image conditioning is off
        )

        # Use the scheduler's `step` function to compute the previous noisy sample
        latent = scheduler.step(noise_pred, t, latent).prev_sample

print("--- Denoising complete. ---")

# --- Decode Latent and Extract Mesh ---
print("--- Decoding Latent and Extracting Mesh ---")
with torch.no_grad():
    decoded_latents = vae.decode(latent)
    mesh_result = vae.extract_geometry(
        decoded_latents, mc_level=0.5, bounds=[-1, -1, -1, 1, 1, 1], octree_resolution=256
    )[0]

final_mesh = trimesh.Trimesh(
    vertices=mesh_result.verts.cpu().numpy(),
    faces=mesh_result.faces.cpu().numpy()
)
final_mesh.export(OUTPUT_FILENAME)

print(f"\n--- ✨ Success! Mesh saved to {OUTPUT_FILENAME} ---")



In [ ]:
package_dirs = [
    "./Step1X-3D/step1x3d_geometry",
    "./Step1X-3D/step1x3d_geometry/utils",
    "./Step1X-3D/step1x3d_geometry/models",
    "./Step1X-3D/step1x3d_geometry/models/pipelines",
    "./Step1X-3D/step1x3d_texture",
    "./Step1X-3D/step1x3d_texture/utils",
    "./Step1X-3D/step1x3d_texture/pipelines",
    "./Step1X-3D/step1x3d_texture/differentiable_renderer",
]

for pkg_dir in package_dirs:
    os.makedirs(pkg_dir, exist_ok=True)
    init_path = os.path.join(pkg_dir, "__init__.py")
    if not os.path.exists(init_path):
        with open(init_path, 'w') as f:
            pass
        print(f"Created: {init_path}")
print("✅ Package structure initialized successfully.\n")



from step1x3d_geometry.models.pipelines.pipeline_utils import (remove_floater,remove_degenerate_face,reduce_face,)
final_mesh = remove_floater(final_mesh)
print("-> Floaters removed.")
final_mesh = remove_degenerate_face(final_mesh)
print("-> Degenerate faces removed.")
final_mesh = reduce_face(final_mesh, max_facenum=200000)
print("-> Face count reduced.")
final_mesh = final_mesh.smooth_shaded
print("-> Smooth shading applied.")
final_mesh.export(OUTPUT_FILENAME)
print(f"\n--- ✅ Success! Cleaned mesh saved to {OUTPUT_FILENAME} ---")

# =========================================================================================
# TRAIN TEXTURE GENERATION MODEL: DOWNLOAD DATASET & TRAIN THE MODEL
# This script download the dataset then process them and lastly train the Texture gen model.
# =========================================================================================

In [3]:
pip install trimesh pyrender xatlas opencv-python torch scipy xatlas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.7 MB/s eta 0:00:00
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745193 sha256=92864f36c7db54ea4dae5ce9dd8b370f8c4ec7f26ed0dcb1f35f8d0928ad63fa
  Stored in directory: /root/.cache/pip/wheels/2f/37/f5/f88cd3dddf75bc3ce608e44bf8a79078c408bf1f351a50818e
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.9
    Uninstalling PyOpenGL-3.1.9:
      Successfully uninstalled PyOpenGL-3.1.9
Note: you may need to restart the kernel to use updated packages.


In [6]:
package_dirs = [
    "./Step1X-3D/step1x3d_geometry",
    "./Step1X-3D/step1x3d_geometry/utils",
    "./Step1X-3D/step1x3d_geometry/models",
    "./Step1X-3D/step1x3d_geometry/models/pipelines",
    "./Step1X-3D/step1x3d_texture",
    "./Step1X-3D/step1x3d_texture/utils",
    "./Step1X-3D/step1x3d_texture/pipelines",
    "./Step1X-3D/step1x3d_texture/differentiable_renderer",
]

for pkg_dir in package_dirs:
    os.makedirs(pkg_dir, exist_ok=True)
    init_path = os.path.join(pkg_dir, "__init__.py")
    if not os.path.exists(init_path):
        with open(init_path, 'w') as f:
            pass
        print(f"Created: {init_path}")
print("✅ Package structure initialized successfully.\n")


Created: ./Step1X-3D/step1x3d_geometry/models/pipelines/__init__.py
Created: ./Step1X-3D/step1x3d_texture/__init__.py
Created: ./Step1X-3D/step1x3d_texture/pipelines/__init__.py
✅ Package structure initialized successfully.



In [7]:
!find /kaggle/ -name "custom_rasterizer"
%cd /kaggle/working/Step1X-3D/step1x3d_texture/custom_rasterizer
!pip install .
%cd /kaggle/working/

/kaggle/working/Step1X-3D/step1x3d_texture/custom_rasterizer
/kaggle/working/Step1X-3D/step1x3d_texture/custom_rasterizer/custom_rasterizer
/kaggle/working/Step1X-3D/step1x3d_texture/custom_rasterizer
Processing /kaggle/working/Step1X-3D/step1x3d_texture/custom_rasterizer
  Preparing metadata (setup.py) ... done
  Created wheel for custom_rasterizer: filename=custom_rasterizer-0.1-cp311-cp311-linux_x86_64.whl size=4602153 sha256=7c7cae8b40260ab2973dcb5d6242893a2c7875eaf76aef3f107d29ab046cd4df
  Stored in directory: /tmp/pip-ephem-wheel-cache-j6uh33no/wheels/9d/39/73/c2a490a839326da41ea6388e6dfb628e5fac1f2ef34553535f
Successfully built custom_rasterizer
/kaggle/working


In [8]:
import os
import torch
import trimesh
import numpy as np
from PIL import Image
from tqdm import tqdm
import json
import traceback
import xatlas
from step1x3d_texture.utils.render import load_mesh, render, NVDiffRastContextWrapper
from step1x3d_texture.utils.camera import get_orthogonal_camera
from step1x3d_texture.utils.saving import tensor_to_image
from step1x3d_geometry.models.pipelines.pipeline_utils import preprocess_image
from step1x3d_texture.pipelines.step1x_3d_texture_synthesis_pipeline import Step1X3DTexturePipeline, Step1X3DTextureConfig

INPUT_DIR = '/kaggle/working/AI-Game-Engine/testing_dataset'
OUTPUT_DIR = 'processed_texture_data_complete'
RESOLUTION = 768

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

print("Initializing GPU rendering context...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ctx = NVDiffRastContextWrapper(device=device, context_type="cuda")
print(f"Context initialized on {device}.")

def generate_optimized_uvs(mesh):
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump(concatenate=True)

    try:
        vmapping, indices, uvs = xatlas.parametrize(mesh.vertices, mesh.faces)
        new_vertices = mesh.vertices[vmapping]
        new_faces = indices
    
        uv_optimized_mesh = trimesh.Trimesh(vertices=new_vertices, faces=new_faces, process=False)
        uv_optimized_mesh.visual = trimesh.visual.texture.TextureVisuals(uv=uvs)
    except Exception as e:
        print(f"    xAtlas failed with error: {e}. Returning original mesh.")
        return mesh 
    
    return uv_optimized_mesh


elevations = [0, 0, 0, 0, 90, -90]
azimuths = [0, 180, 90, -90, 0, 0] 
view_names = ['front', 'back', 'right', 'left', 'top', 'bottom']

cameras = get_orthogonal_camera(
    elevation_deg=elevations,
    azimuth_deg=azimuths,
    distance=[1.8] * 6,
    left=-0.55, right=0.55, bottom=-0.55, top=0.55,
    device=device
)
camera_poses_dict = {name: cameras.c2w[i].cpu().numpy().tolist() for i, name in enumerate(view_names)}

MAX_FILES = None

glb_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.glb') and os.path.isfile(os.path.join(INPUT_DIR, f))]
if MAX_FILES is not None:
    glb_files = glb_files[:MAX_FILES]
batch_data = []

for glb_filename in tqdm(glb_files, desc="Processing Meshes"):
    glb_path = os.path.join(INPUT_DIR, glb_filename)
    model_id = os.path.splitext(glb_filename)[0]
    
    model_output_dir = os.path.join(OUTPUT_DIR, model_id)
    os.makedirs(model_output_dir, exist_ok=True)

    try:
        raw_mesh = trimesh.load(glb_path, force='mesh', process=False)
        texture_tensor = None
        if (
            hasattr(raw_mesh.visual, "material") and
            hasattr(raw_mesh.visual.material, "baseColorTexture") and
            raw_mesh.visual.material.baseColorTexture is not None
        ):
            texture_image = raw_mesh.visual.material.baseColorTexture
            texture_data = np.array(texture_image.convert("RGB")) / 255.0
            texture_tensor = torch.tensor(texture_data, dtype=torch.float32).to(device)
        else:
            print(f"⚠️  Warning: No texture found for {glb_filename}. Using a default gray texture.")
            texture_tensor = torch.ones((1024, 1024, 3), dtype=torch.float32).to(device) * 0.5
        # === MODIFICATION END ===

        uv_optimized_mesh = generate_optimized_uvs(raw_mesh)
        uv_mesh_path = os.path.join(model_output_dir, "mesh_with_uvs.glb")
        uv_optimized_mesh.export(uv_mesh_path)
        
        mesh_for_render, _ = load_mesh(uv_optimized_mesh, rescale=True, device=device)

        # === MODIFICATION START: Use the texture_override parameter ===
        render_output = render(
            ctx, mesh_for_render, cameras, height=RESOLUTION, width=RESOLUTION,
            render_attr=True, render_normal=True,
            texture_override=texture_tensor
        )
        # === MODIFICATION END ===

        position_maps = (render_output.pos + 0.5).clamp(0, 1)
        normal_maps = (render_output.normal / 2 + 0.5).clamp(0, 1)
        albedo_maps = render_output.attr
        
        output_paths = { 'albedo': {}, 'normal': {}, 'position': {}, 'reference_image': '' }
        
        front_albedo_tensor = albedo_maps[view_names.index('front')]
        front_albedo_pil = tensor_to_image(front_albedo_tensor)

        reference_image_pil = preprocess_image(front_albedo_pil)
        ref_image_path = os.path.join(model_output_dir, "reference_image.png")
        reference_image_pil.save(ref_image_path)
        output_paths['reference_image'] = ref_image_path

        for i, view_name in enumerate(view_names):
            albedo_path = os.path.join(model_output_dir, f"{view_name}_albedo.png")
            tensor_to_image(albedo_maps[i]).save(albedo_path)
            output_paths['albedo'][view_name] = albedo_path
            
            normal_path = os.path.join(model_output_dir, f"{view_name}_normal.png")
            tensor_to_image(normal_maps[i]).save(normal_path)
            output_paths['normal'][view_name] = normal_path
            
            position_path = os.path.join(model_output_dir, f"{view_name}_position.png")
            tensor_to_image(position_maps[i]).save(position_path)
            output_paths['position'][view_name] = position_path

        camera_pose_path = os.path.join(model_output_dir, 'camera_poses.json')
        with open(camera_pose_path, 'w') as f:
            json.dump(camera_poses_dict, f, indent=4)

        record = {
            'model_id': model_id,
            'uv_optimized_mesh_path': uv_mesh_path, #generated texture are baked using this uv_mesh
            'reference_image_path': output_paths['reference_image'], #reference image for evaluation/comparison, its primary role during inference is to be the conditional input that tells the model what style, color, and pattern to generate.
            'albedo_map_paths': output_paths['albedo'], #used for training and it is the output of the model
            'normal_map_paths': output_paths['normal'], #used for training but as condition to the model but also used in inference as input
            'position_map_paths': output_paths['position'], #used for training but as condition to the model but also used in inference as input
            'camera_pose_path': camera_pose_path #Will be used as instruction for generation pipeline during inference
        }
        batch_data.append(record)

    except Exception as e:
        print(f"Failed to process {glb_filename}: {e}")
        traceback.print_exc()

batch_file_path = os.path.join(OUTPUT_DIR, 'dataset_manifest.json')
with open(batch_file_path, 'w') as f:
    json.dump(batch_data, f, indent=4)

print(f"\n✅ Processing complete. {len(batch_data)} assets processed.")
print(f"Processed data saved to: {OUTPUT_DIR}")
print(f"Dataset manifest saved to: {batch_file_path}")

Initializing GPU rendering context...


/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Context initialized on cuda.


Processing Meshes:   0%|          | 0/108 [00:00<?, ?it/s]/kaggle/working/./Step1X-3D/step1x3d_texture/utils/render.py:87: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /pytorch/aten/src/ATen/native/Cross.cpp:62.)
  face_normals = torch.cross(v1 - v0, v2 - v0)

  0%|                                               | 0.00/176M [00:00<?, ?B/s]
  4%|█▍                                    | 6.41M/176M [00:00<00:02, 64.1MB/s]
  8%|███▏                                  | 14.8M/176M [00:00<00:02, 75.6MB/s]
 13%|████▉                                 | 23.0M/176M [00:00<00:01, 78.6MB/s]
 18%|██████▋                               | 30.9M/176M [00:00<00:01, 77.3MB/s]
 22%|████████▎                             | 38.6M/176M [00:00<00:01, 76.7MB/s]
 26%|██████████                      

⚠️  Warning: No texture found for 2011_porsche_911_speedster_997-2.glb. Using a default gray texture.


...

⚠️  Warning: No texture found for ak-47.glb. Using a default gray texture.


Processing Meshes:  80%|███████▉  | 86/108 [22:35<24:07, 65.79s/it]   

⚠️  Warning: No texture found for tomos_atx_50c.glb. Using a default gray texture.


Processing Meshes: 100%|██████████| 108/108 [25:21<00:00, 14.09s/it]


✅ Processing complete. 108 assets processed.
Processed data saved to: processed_texture_data_complete
Dataset manifest saved to: processed_texture_data_complete/dataset_manifest.json


In [ ]:
import os
import json
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm.auto import tqdm
from accelerate import Accelerator
from diffusers import AutoencoderKL, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from torchvision import transforms
import numpy as np
from mmfreelm.models.hgrn_bit.texture_dit import TernaryMVAdapter_models
model_variant='S'
CONFIG = {
    "dataset_manifest": "./processed_texture_data_complete/dataset_manifest.json",
    "output_dir": "./texture_model_output",
    "image_resolution": 768,
    "latent_resolution": 96,
    "train_batch_size": 1,
    "num_train_epochs": 3, #Number of epoch
    "learning_rate": 1e-4,
    "adam_beta1": 0.9,
    "adam_beta2": 0.999,
    "adam_weight_decay": 1e-2,
    "adam_epsilon": 1e-08,
    "mixed_precision": "fp16",
    "gradient_accumulation_steps": 4,
    "save_steps": 1000,
    "num_views": 6,
    "vae_model_id": "madebyollin/sdxl-vae-fp16-fix",
    "text_encoder_id": "stabilityai/stable-diffusion-xl-base-1.0",
}

class TextureDataset(Dataset):
    """
    Dataset to load the pre-processed texture generation data.
    """
    def __init__(self, manifest_path, resolution):
        print(f"Loading dataset manifest from: {manifest_path}")
        with open(manifest_path, 'r') as f:
            self.manifest = json.load(f)
        print(f"Found {len(self.manifest)} 3D assets.")

        self.transform = transforms.Compose([
            transforms.Resize((resolution, resolution), interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]) # Normalize to [-1, 1]
        ])
        self.control_transform = transforms.Compose([
            transforms.Resize((resolution, resolution), interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.manifest)

    def __getitem__(self, idx):
        item = self.manifest[idx]
        view_names = ['front', 'back', 'right', 'left', 'top', 'bottom']

        albedos = [Image.open(item['albedo_map_paths'][v]).convert("RGB") for v in view_names]
        albedos_tensor = torch.stack([self.transform(img) for img in albedos])

        normals = [Image.open(item['normal_map_paths'][v]).convert("RGB") for v in view_names]
        positions = [Image.open(item['position_map_paths'][v]).convert("RGB") for v in view_names]

        normals_tensor = torch.stack([self.control_transform(img) for img in normals])
        positions_tensor = torch.stack([self.control_transform(img) for img in positions])

        control_images = torch.cat([normals_tensor, positions_tensor], dim=1) # Shape: (NumViews, 6, H, W)

        ref_image = Image.open(item['reference_image_path']).convert("RGB")
        ref_image_tensor = self.transform(ref_image)

        text_prompt = "a high-quality texture"

        return {
            "albedos": albedos_tensor,
            "control_images": control_images,
            "reference_image": ref_image_tensor,
            "prompt": text_prompt,
        }


def main():
    accelerator = Accelerator(
        gradient_accumulation_steps=CONFIG["gradient_accumulation_steps"],
        mixed_precision=CONFIG["mixed_precision"],
        log_with="tensorboard",
        project_dir=os.path.join(CONFIG["output_dir"], "logs")
    )

    vae = AutoencoderKL.from_pretrained(CONFIG["vae_model_id"], torch_dtype=torch.float16)
    tokenizer = CLIPTokenizer.from_pretrained(CONFIG["text_encoder_id"], subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(CONFIG["text_encoder_id"], subfolder="text_encoder", torch_dtype=torch.float16)
    
    noise_scheduler = DDPMScheduler.from_pretrained(CONFIG["text_encoder_id"], subfolder="scheduler")
    model_constructor = TernaryMVAdapter_models[f'TernaryMVAdapter-{model_variant}']
    
    print(f"Instantiating model: TernaryMVAdapter-{model_variant}")
    model = model_constructor(
        input_size=CONFIG["latent_resolution"],
        patch_size=2,
        in_channels=4, 
        cond_channels=6, 
        text_embed_dim=768,
        learn_sigma=True,
        full_precision = False,
        optimized_bitlinear: bool = False,

    )

    vae.requires_grad_(False)
    text_encoder.requires_grad_(False)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=CONFIG["learning_rate"],
        betas=(CONFIG["adam_beta1"], CONFIG["adam_beta2"]),
        weight_decay=CONFIG["adam_weight_decay"],
        eps=CONFIG["adam_epsilon"],
    )

    train_dataset = TextureDataset(CONFIG["dataset_manifest"], CONFIG["image_resolution"])
    train_dataloader = DataLoader(train_dataset, batch_size=CONFIG["train_batch_size"], shuffle=True)

    model, optimizer, train_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader
    )
    
    vae.to(accelerator.device)
    text_encoder.to(accelerator.device)

    global_step = 0
    
    print("🚀 Starting training...")
    for epoch in range(CONFIG["num_train_epochs"]):
        progress_bar = tqdm(total=len(train_dataloader), desc=f"Epoch {epoch+1}/{CONFIG['num_train_epochs']}")
        for step, batch in enumerate(train_dataloader):
            batch_size = batch["albedos"].shape[0]
            clean_images = batch["albedos"].view(-1, 3, CONFIG["image_resolution"], CONFIG["image_resolution"])
            control_images = batch["control_images"].view(-1, 6, CONFIG["image_resolution"], CONFIG["image_resolution"])
            ref_images = batch["reference_image"]
            with torch.no_grad():
                clean_latents = vae.encode(clean_images.to(dtype=torch.float16)).latent_dist.sample() * vae.config.scaling_factor
                ref_latents = vae.encode(ref_images.to(dtype=torch.float16)).latent_dist.sample() * vae.config.scaling_factor
                text_inputs = tokenizer(batch["prompt"], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
                prompt_embeds = text_encoder(text_inputs.input_ids.to(accelerator.device))[0]
            noise = torch.randn_like(clean_latents)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (clean_latents.shape[0],), device=accelerator.device)
            noisy_latents = noise_scheduler.add_noise(clean_latents, noise, timesteps)
            with accelerator.accumulate(model):
                model_output = model(
                    x=noisy_latents,
                    t=timesteps,
                    num_views=CONFIG["num_views"],
                    encoder_hidden_states=prompt_embeds,
                    control_image_feature=control_images,
                    ref_hidden_states=ref_latents
                )

                unwrapped_model = accelerator.unwrap_model(model)
                if unwrapped_model.learn_sigma:
                    noise_pred, _ = model_output.chunk(2, dim=1)
                else:
                    noise_pred = model_output
                loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")
                
                accelerator.backward(loss)
                optimizer.step()
                optimizer.zero_grad()
            progress_bar.update(1)
            progress_bar.set_postfix(loss=loss.detach().item())
            global_step += 1

            if global_step % CONFIG["save_steps"] == 0:
                # Save a checkpoint
                save_path = os.path.join(CONFIG["output_dir"], f"checkpoint-{global_step}")
                accelerator.save_state(save_path)
                print(f"✅ Saved checkpoint to {save_path}")

    print("✅ Training complete!")
    print("💾 Saving final model state...")
    accelerator.wait_for_everyone()
    save_path = os.path.join(CONFIG["output_dir"], "final_model")
    accelerator.save_state(save_path)
    print(f"✅ Final model saved to {save_path}")

if __name__ == "__main__":
    os.makedirs(CONFIG["output_dir"], exist_ok=True)
    main()

In [ ]:
import os
import torch
import trimesh
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
from torchvision import transforms
from step1x3d_geometry.models.pipelines.pipeline_utils import reduce_face, remove_degenerate_face
from step1x3d_texture.pipelines.step1x_3d_texture_synthesis_pipeline import Step1X3DTexturePipeline
from step1x3d_texture.utils.render import NVDiffRastContextWrapper, load_mesh as load_mesh_for_render, render
from step1x3d_texture.utils.camera import get_orthogonal_camera
from step1x3d_texture.utils.saving import tensor_to_image

from mmfreelm.models.hgrn_bit.texture_dit import TernaryMVAdapter_models
from diffusers import AutoencoderKL, DDPMScheduler
from accelerate import Accelerator
INPUT_MESH_PATH = "/kaggle/working/AI-Game-Engine/testing_dataset/axe.glb"
REFERENCE_IMAGE_PATH = "/kaggle/working/processed_texture_data_complete/axe/reference_image.png" 
OUTPUT_MESH_PATH = "./test_textured.glb"

YOUR_MODEL_CHECKPOINT_DIR = "./texture_model_output/final_model" 

NUM_VIEWS = 6
IMAGE_RESOLUTION = 768
GUIDANCE_SCALE = 3.0
NUM_INFERENCE_STEPS = 50

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16

# =================================================================
# ============== 2.LOAD AND CLEAN THE INPUT MESH ==================                                
# =================================================================
print("--- 🧼 Stage 1: Loading and Cleaning Mesh ---")

if not os.path.exists(INPUT_MESH_PATH):
    raise FileNotFoundError(f"Input mesh not found at {INPUT_MESH_PATH}")
raw_mesh = trimesh.load(INPUT_MESH_PATH)
print(f"-> Loaded raw mesh from {INPUT_MESH_PATH}")

cleaned_mesh = remove_degenerate_face(raw_mesh)
cleaned_mesh = reduce_face(cleaned_mesh)
print("-> Mesh cleaned successfully (degenerate faces removed, face count reduced).")

# =================================================================
# 3. GENERATE MULTI-VIEW IMAGES WITH YOUR CUSTOM MODEL
# =================================================================
print("\n--- 🖼️ Stage 2: Generating Multi-View Images with Your TernaryMVAdapter ---")

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=dtype).to(device)
scheduler = DDPMScheduler.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="scheduler")
scheduler.set_timesteps(NUM_INFERENCE_STEPS)

model_constructor = TernaryMVAdapter_models[f'TernaryMVAdapter-S']
model = model_constructor(
    input_size=IMAGE_RESOLUTION // 8, 
    patch_size=2,
    in_channels=4,
    cond_channels=6,
    text_embed_dim=768,
    learn_sigma=True,
    optimized_bitlinear: bool = False,
    full_precision: bool = False,
)


accelerator = Accelerator()
model = accelerator.prepare(model)
accelerator.load_state(YOUR_MODEL_CHECKPOINT_DIR)
model = accelerator.unwrap_model(model).to(device, dtype=dtype).eval()
print(f"-> Successfully loaded your custom model from {YOUR_MODEL_CHECKPOINT_DIR}")

print("-> Rendering Normal and Position maps for conditioning...")
cameras = get_orthogonal_camera(
    elevation_deg=[0, 0, 0, 0, 89.99, -89.99],
    distance=[1.8] * NUM_VIEWS, left=-0.55, right=0.55, bottom=-0.55, top=0.55,
    azimuth_deg=[x - 90 for x in [0, 90, 180, 270, 180, 180]],
    device=device,
)
ctx = NVDiffRastContextWrapper(device=device, context_type="cuda")
mesh_for_render, _ = load_mesh_for_render(cleaned_mesh, rescale=True, device=device)
render_out = render(ctx, mesh_for_render, cameras, height=IMAGE_RESOLUTION, width=IMAGE_RESOLUTION, render_attr=False)
control_images = torch.cat([
    (render_out.pos + 0.5).clamp(0, 1),
    (render_out.normal / 2 + 0.5).clamp(0, 1),
], dim=-1).permute(0, 3, 1, 2).to(device, dtype=dtype)

ref_transform = transforms.Compose([
    transforms.Resize((IMAGE_RESOLUTION, IMAGE_RESOLUTION), interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
ref_image = Image.open(REFERENCE_IMAGE_PATH).convert("RGB")
ref_image_tensor = ref_transform(ref_image).unsqueeze(0).to(device, dtype=dtype)
with torch.no_grad():
    ref_latents = vae.encode(ref_image_tensor).latent_dist.sample() * vae.config.scaling_factor

# --- Prepare Latents and Prompt Embeds (Placeholder) ---
latents = torch.randn((NUM_VIEWS, 4, IMAGE_RESOLUTION // 8, IMAGE_RESOLUTION // 8), device=device, dtype=dtype)

# Using a placeholder since your model doesn't rely heavily on text
prompt_embeds = torch.zeros((1, 77, 768), device=device, dtype=dtype) # Dummy embeds, corrected shape

print("-> Starting denoising loop...")
with torch.no_grad():
    for t in tqdm(scheduler.timesteps):
        # Your model's forward pass for inference
        noise_pred = model(
            x=latents, 
            t=t.expand(NUM_VIEWS).to(device), # <-- ADD .to(device) HERE
            num_views=NUM_VIEWS,
            encoder_hidden_states=prompt_embeds, 
            control_image_feature=control_images,
            ref_hidden_states=ref_latents
        )
        if model.learn_sigma:
            noise_pred, _ = noise_pred.chunk(2, dim=1)
        
        # Scheduler step
        latents = scheduler.step(noise_pred, t, latents).prev_sample.to(dtype)


with torch.no_grad():
    latents = 1 / vae.config.scaling_factor * latents
    # Cast the latents back to the expected float16 dtype
    decoded_images = vae.decode(latents.to(dtype)).sample
    decoded_images = (decoded_images / 2 + 0.5).clamp(0, 1)

generated_views = [transforms.ToPILImage()(img) for img in decoded_images]
print("-> Multi-view image generation complete.")


print("\n--- 🎨 Stage 3: Baking and Inpainting Texture ---")

official_pipeline = Step1X3DTexturePipeline.from_pretrained(
    "stepfun-ai/Step1X-3D", subfolder="Step1X-3D-Texture"
)

print("-> Performing UV unwrapping with xatlas...")
unwrapped_mesh = official_pipeline.mesh_uv_wrap(cleaned_mesh)
official_pipeline.mesh_render.load_mesh(unwrapped_mesh, auto_center=False, scale_factor=1.0)

print("-> Baking generated views onto UV map...")
texture, mask = official_pipeline.bake_from_multiview(
    official_pipeline.mesh_render,
    generated_views,
    official_pipeline.config.selected_camera_elevs,
    official_pipeline.config.selected_camera_azims,
    official_pipeline.config.selected_view_weights,
    method="fast",
)
mask_np = (mask.squeeze(-1).cpu().numpy() * 255).astype(np.uint8)

print("-> Inpainting texture to fill holes...")
texture_tensor = official_pipeline.texture_inpaint(official_pipeline.mesh_render, texture, mask_np)

official_pipeline.mesh_render.set_texture(texture_tensor)
final_textured_mesh = official_pipeline.mesh_render.save_mesh()

# =================================================================
# 5. EXPORT FINAL RESULT
# =================================================================
print("\n--- ✅ Stage 4: Exporting Final Textured Mesh ---")
final_textured_mesh.export(OUTPUT_MESH_PATH)
print(f"-> Final textured mesh saved to {OUTPUT_MESH_PATH}")

# =========================================================================================
# TRAIN ANIMATION GENERATION MODEL: DOWNLOAD DATASET & TRAIN THE MODEL
# This script download the dataset then process them and lastly train the animation gen model.
# =========================================================================================

In [ ]:
import os
import json
import torch
import traceback
import numpy as np
import trimesh
from tqdm import tqdm
import logging
import random
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, Trainer, TrainingArguments
from mmfreelm.models.hgrn_bit.animation import AnimDiT_models  # Assuming the animation.py is saved as animation.py
from diffusion_model import GaussianDiffusion, ModelMeanType, ModelVarType, LossType, get_named_beta_schedule, _extract_into_tensor
from step1x3d_geometry.models.pipelines.pipeline import Step1X3DGeometryPipeline
from safetensors.torch import load_file as safetensors_load
import shutil
from pygltflib import GLTF2
from scipy.spatial.transform import Rotation as R


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32
print(f"Using device: {device}, dtype: {dtype}")


In [ ]:
print("Initializing Step1X-3D VAE...")
try:
    geometry_pipeline = Step1X3DGeometryPipeline.from_pretrained(
        "stepfun-ai/Step1X-3D",
        subfolder='Step1X-3D-Geometry-1300m',
        torch_dtype=dtype
    )
    vae = geometry_pipeline.vae.to(device)
    vae.eval()
    print("Step1X-3D VAE initialized successfully.")
except Exception as e:
    print(f"Error initializing pipeline: {e}")
    vae = None

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer loaded.")


In [ ]:
def load_local_dataset(dataset_dir):
    json_path = os.path.join(dataset_dir, "dataset.json")
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"dataset.json not found in {dataset_dir}")

    with open(json_path) as f:
        metadata = json.load(f)

    glb_paths = []
    texts = []

    logger.info(f"Loading data from {dataset_dir}...")

    for glb_file, info in metadata.items():
        glb_path = os.path.join(dataset_dir, glb_file)
        if not os.path.exists(glb_path):
            logger.warning(f"File not found, skipping: {glb_path}")
            continue

        # Replace None or empty strings with 'Unknown'
        category = info.get("category") or "Unknown"
        make = info.get("make") or "Unknown"
        model = info.get("model") or "Unknown"
        year = info.get("year") or "Unknown"
        description = info.get("description") or "No description"
        tags = ", ".join(info.get("tags", [])) if info.get("tags") else "None"

        merged_text = (
            f"category: {category} "
            f"make: {make} "
            f"model: {model} "
            f"year: {year} "
            f"description: {description} "
            f"tags: {tags}"
        )

        glb_paths.append(glb_path)
        texts.append(merged_text)

    logger.info(f"Found {len(glb_paths)} valid GLB files.")
    if glb_paths:
        logger.info("Example GLB: %s", glb_paths[0])
        logger.info("Example text: %s", texts[0][:500])

    return glb_paths, texts
    

In [ ]:
def find_all_mesh_nodes(gltf, node_indices):
    """Helper function to find ALL nodes with meshes, including children."""
    mesh_nodes = []
    for node_index in node_indices:
        node = gltf.nodes[node_index]
        if node.mesh is not None:
            mesh_nodes.append(node_index)
        if node.children:
            mesh_nodes.extend(find_all_mesh_nodes(gltf, node.children))
    return mesh_nodes


def extract_and_combine_frame_meshes(glb_path, num_frames=10):
    """
    Loads a GLB file, calculates the animated state of all its meshes (including skinning and morphs)
    at different time steps, and returns a list of combined trimesh.Trimesh objects for each frame.
    """
    try:
        logger.info(f"Processing {glb_path} for frame extraction...")
        gltf = GLTF2().load(glb_path)

        scene_nodes = gltf.scenes[gltf.scene].nodes
        all_mesh_node_indices = find_all_mesh_nodes(gltf, scene_nodes)

        if not all_mesh_node_indices:
            logger.error(f"Could not find any node with a mesh in {glb_path}")
            return None

        base_primitives_data = []
        node_defaults = {}
        parents = {i: None for i in range(len(gltf.nodes))}

        for i, node in enumerate(gltf.nodes):
            if node.matrix:
                base_transform = np.array(node.matrix).reshape(4, 4)
            else:
                scale = np.array(node.scale if node.scale else [1.0, 1.0, 1.0])
                rotation = np.array(node.rotation if node.rotation else [0.0, 0.0, 0.0, 1.0])
                translation = np.array(node.translation if node.translation else [0.0, 0.0, 0.0])
                
                scale_mat = np.diag(np.append(scale, 1.0))
                rot_mat = np.eye(4); rot_mat[:3, :3] = R.from_quat(rotation).as_matrix()
                trans_mat = np.eye(4); trans_mat[:3, 3] = translation
                base_transform = trans_mat @ rot_mat @ scale_mat

            node_defaults[f"node_{i}_base_transform"] = base_transform
            for child in node.children or []: parents[child] = i

            if node.mesh is not None:
                mesh_gltf = gltf.meshes[node.mesh]
                max_morphs = max(len(p.targets or []) for p in mesh_gltf.primitives) if mesh_gltf.primitives else 0
                default_weights = np.array(mesh_gltf.weights) if mesh_gltf.weights else np.zeros(max_morphs, dtype=np.float32)
                node_defaults[f"node_{i}_weights"] = default_weights

                is_skinned = node.skin is not None
                skin = gltf.skins[node.skin] if is_skinned else None
                joints = skin.joints if is_skinned else []
                ibms = None
                if is_skinned and skin.inverseBindMatrices is not None:
                    ibm_accessor = gltf.accessors[skin.inverseBindMatrices]
                    ibm_view = gltf.bufferViews[ibm_accessor.bufferView]
                    ibms = np.frombuffer(gltf.binary_blob(), dtype=np.float32, count=ibm_accessor.count * 16, offset=ibm_view.byteOffset + ibm_accessor.byteOffset).reshape(-1, 4, 4)
                else:
                    ibms = [np.eye(4) for _ in joints] if is_skinned else []
                
                for prim_idx, primitive in enumerate(mesh_gltf.primitives):
                    pos_accessor = gltf.accessors[primitive.attributes.POSITION]
                    pos_view = gltf.bufferViews[pos_accessor.bufferView]
                    positions = np.frombuffer(gltf.binary_blob(), dtype=np.float32, count=pos_accessor.count * 3, offset=pos_view.byteOffset + pos_accessor.byteOffset).reshape(-1, 3)

                    face_accessor = gltf.accessors[primitive.indices]
                    face_view = gltf.bufferViews[face_accessor.bufferView]
                    face_dtype = np.uint16 if face_accessor.componentType == 5123 else np.uint32
                    faces = np.frombuffer(gltf.binary_blob(), dtype=face_dtype, count=face_accessor.count, offset=face_view.byteOffset + face_accessor.byteOffset).reshape(-1, 3)
                    
                    morph_deltas = []
                    if primitive.targets:
                        for target in primitive.targets:
                            if target.POSITION is not None:
                                delta_accessor = gltf.accessors[target.POSITION]
                                delta_view = gltf.bufferViews[delta_accessor.bufferView]
                                deltas = np.frombuffer(gltf.binary_blob(), dtype=np.float32, count=delta_accessor.count*3, offset=delta_view.byteOffset + delta_accessor.byteOffset).reshape(-1,3)
                                morph_deltas.append(deltas)
                    
                    joints_data, vertex_weights = None, None
                    if is_skinned:
                        if hasattr(primitive.attributes, 'JOINTS_0') and primitive.attributes.JOINTS_0 is not None:
                            j_accessor = gltf.accessors[primitive.attributes.JOINTS_0]
                            j_view = gltf.bufferViews[j_accessor.bufferView]
                            j_dtype = {5121: np.uint8, 5123: np.uint16}.get(j_accessor.componentType, np.uint32)
                            joints_data = np.frombuffer(gltf.binary_blob(), dtype=j_dtype, count=j_accessor.count*4, offset=j_view.byteOffset+j_accessor.byteOffset).reshape(-1,4)
                        if hasattr(primitive.attributes, 'WEIGHTS_0') and primitive.attributes.WEIGHTS_0 is not None:
                            w_accessor = gltf.accessors[primitive.attributes.WEIGHTS_0]
                            w_view = gltf.bufferViews[w_accessor.bufferView]
                            vertex_weights = np.frombuffer(gltf.binary_blob(), dtype=np.float32, count=w_accessor.count*4, offset=w_view.byteOffset+w_accessor.byteOffset).reshape(-1,4)

                    base_primitives_data.append({
                        'node_idx': i, 'prim_idx': prim_idx, 'base_positions': positions, 'faces': faces, 'morph_deltas': morph_deltas,
                        'is_skinned': is_skinned, 'joints_list': joints, 'ibms': ibms, 'joints': joints_data, 'vertex_weights': vertex_weights
                    })

        def get_world_transform(node_idx, current_locals, memo):
            if node_idx in memo: return memo[node_idx]
            parent = parents[node_idx]
            local = current_locals[node_idx]
            if parent is None: memo[node_idx] = local
            else: memo[node_idx] = get_world_transform(parent, current_locals, memo) @ local
            return memo[node_idx]

        def build_mesh_at_time(t):
            num_nodes = len(gltf.nodes)
            current_locals = {idx: node_defaults[f"node_{idx}_base_transform"].copy() for idx in range(num_nodes)}
            current_weights = {idx: node_defaults[f"node_{idx}_weights"].copy() for idx in range(num_nodes) if f"node_{idx}_weights" in node_defaults}
            
            if gltf.animations:
                animation = gltf.animations[0]
                for channel in animation.channels:
                    if channel.target.node is None: continue
                    sampler = animation.samplers[channel.sampler]
                    
                    input_accessor = gltf.accessors[sampler.input]
                    times = np.frombuffer(gltf.binary_blob(), dtype=np.float32, count=input_accessor.count, offset=gltf.bufferViews[input_accessor.bufferView].byteOffset + input_accessor.byteOffset)
                    
                    output_accessor = gltf.accessors[sampler.output]
                    num_comp = {"SCALAR": 1, "VEC3": 3, "VEC4": 4}[output_accessor.type]
                    values = np.frombuffer(gltf.binary_blob(), dtype=np.float32, count=output_accessor.count * num_comp, offset=gltf.bufferViews[output_accessor.bufferView].byteOffset + output_accessor.byteOffset).reshape(-1, num_comp)
                    
                    if channel.target.path == "weights":
                        num_morphs = len(current_weights.get(channel.target.node, []))
                        if num_morphs > 0: values = values.reshape(-1, num_morphs)

                    idx = np.searchsorted(times, t, side='right')
                    if idx == 0: val = values[0]
                    elif idx >= len(times): val = values[-1]
                    else:
                        t_prev, t_next = times[idx - 1], times[idx]
                        frac = (t - t_prev) / (t_next - t_prev) if (t_next - t_prev) > 0 else 0
                        if sampler.interpolation == "STEP": frac = 0
                        val = (1 - frac) * values[idx - 1] + frac * values[idx]
                    
                    node_idx = channel.target.node
                    path = channel.target.path
                    if path == "translation": current_locals[node_idx][:3, 3] = val
                    elif path == "rotation": current_locals[node_idx][:3, :3] = R.from_quat(val).as_matrix()
                    elif path == "scale": current_locals[node_idx][:3, :3] *= val[:, np.newaxis] # Simplified for uniform scale
                    elif path == "weights": current_weights[node_idx] = val

            frame_meshes = []
            memo = {}
            for prim_data in base_primitives_data:
                node_idx = prim_data['node_idx']
                morphed_pos = prim_data['base_positions'].copy()
                if prim_data['morph_deltas']:
                    weights = current_weights.get(node_idx, [])
                    for j in range(min(len(prim_data['morph_deltas']), len(weights))):
                        morphed_pos += weights[j] * prim_data['morph_deltas'][j]

                if prim_data['is_skinned'] and prim_data['joints'] is not None and prim_data['vertex_weights'] is not None:
                    palette = np.array([get_world_transform(j, current_locals, memo) @ ibm for j, ibm in zip(prim_data['joints_list'], prim_data['ibms'])])
                    
                    num_verts = len(morphed_pos)
                    vertices = np.zeros((num_verts, 3), dtype=np.float32)
                    pos_hom = np.hstack((morphed_pos, np.ones((num_verts, 1))))
                    for b in range(4):
                        joint_indices = prim_data['joints'][:, b]
                        weights_b = prim_data['vertex_weights'][:, b]
                        selected_mats = palette[joint_indices]
                        transformed_hom = np.einsum('vab,vb->va', selected_mats, pos_hom)
                        vertices += (transformed_hom[:, :3] * weights_b[:, np.newaxis])
                else: # Non-skinned, just apply node transform
                    transform_matrix = get_world_transform(node_idx, current_locals, memo)
                    pos_hom = np.hstack((morphed_pos, np.ones((len(morphed_pos), 1))))
                    transformed_pos_hom = (transform_matrix @ pos_hom.T).T
                    vertices = transformed_pos_hom[:, :3]

                frame_meshes.append(trimesh.Trimesh(vertices=vertices, faces=prim_data['faces']))
            
            # Combine all mesh parts for this frame into a single mesh
            if not frame_meshes: return None
            return trimesh.util.concatenate(frame_meshes)

        if not gltf.animations:
            logger.warning(f"No animations in {glb_path}. Replicating static mesh.")
            static_mesh = build_mesh_at_time(0.0)
            return [static_mesh.copy() for _ in range(num_frames)] if static_mesh else None

        animation = gltf.animations[0]
        max_time = 0.0
        if animation.samplers:
            max_time = max((gltf.accessors[s.input].max[0] for s in animation.samplers if gltf.accessors[s.input].max), default=0.0)

        if max_time <= 1e-6:
            logger.warning(f"Animation has zero duration in {glb_path}. Replicating first frame.")
            static_mesh = build_mesh_at_time(0.0)
            return [static_mesh.copy() for _ in range(num_frames)] if static_mesh else None
            
        time_steps = np.linspace(0, max_time, num_frames)
        
        all_frame_meshes = []
        for t in time_steps:
            combined_mesh = build_mesh_at_time(t)
            if combined_mesh is None:
                logger.error(f"Failed to build mesh at time {t} for {glb_path}")
                return None
            all_frame_meshes.append(combined_mesh)
            
        logger.info(f"Successfully extracted {len(all_frame_meshes)} combined frame meshes for {glb_path}")
        return all_frame_meshes

    except Exception as e:
        logger.error(f"Error extracting frames from {glb_path}: {str(e)}\n{traceback.format_exc()}")
        return None
# <<< END: NEW MESH EXTRACTION LOGIC >>>

In [ ]:
def sample_uniform_and_sharp_points(mesh, num_uniform_points=16384, num_sharp_points=16384, sharp_threshold_deg=60):
    try:
        # It's crucial to use mesh.area to avoid issues with tiny or degenerate meshes
        if mesh.area < 1e-6:
             logger.warning("Mesh area is near zero. Sampling might fail or be inaccurate.")
             # Fallback: create a small sphere to sample from to avoid crashing
             mesh = trimesh.creation.icosphere(subdivisions=2, radius=0.5)

        logger.debug(f"Sampling {num_uniform_points} uniform and {num_sharp_points} sharp points")
        uniform_points, face_indices_uniform = trimesh.sample.sample_surface(mesh, num_uniform_points)
        uniform_normals = mesh.face_normals[face_indices_uniform]
        sharp_points, sharp_normals = None, None

        edge_angles = mesh.face_adjacency_angles
        sharp_threshold_rad = np.deg2rad(sharp_threshold_deg)
        sharp_edge_indices = np.where(edge_angles > sharp_threshold_rad)[0]

        if len(sharp_edge_indices) > 0:
            face_indices_of_sharp_edges = mesh.face_adjacency[sharp_edge_indices].flatten()
            sharp_face_indices = np.unique(face_indices_of_sharp_edges)
            if len(sharp_face_indices) > 0:
                sharp_mesh = mesh.submesh([sharp_face_indices], append=True)
                if sharp_mesh.vertices.shape[0] > 3 and sharp_mesh.faces.shape[0] > 1 and sharp_mesh.area > 1e-6:
                    sharp_points, face_indices_sharp = trimesh.sample.sample_surface(sharp_mesh, num_sharp_points)
                    sharp_normals = sharp_mesh.face_normals[face_indices_sharp]

        if sharp_points is None or sharp_normals is None:
            logger.debug("No sharp regions found or submesh failed. Using uniform samples for sharp set.")
            sharp_points, face_indices_sharp = trimesh.sample.sample_surface(mesh, num_sharp_points)
            sharp_normals = mesh.face_normals[face_indices_sharp]

        logger.debug(f"Sampled uniform_points shape: {uniform_points.shape}, sharp_points shape: {sharp_points.shape}")
        return (uniform_points, uniform_normals), (sharp_points, sharp_normals)
    except Exception as e:
        logger.error(f"Error in point sampling: {str(e)}\n{traceback.format_exc()}")
        return None


In [ ]:
def process_mesh_to_vae_input(mesh, num_points=32768):
    try:
        logger.debug(f"Processing mesh with {len(mesh.vertices)} vertices, {len(mesh.faces)} faces")
        if not isinstance(mesh, trimesh.Trimesh) or len(mesh.vertices) < 3 or len(mesh.faces) < 1:
            logger.error(f"Invalid mesh: vertices={len(mesh.vertices)}, faces={len(mesh.faces)}")
            return None

        # Fix potential issues with the mesh that can break processing
        mesh.update_faces(mesh.nondegenerate_faces())
        mesh.remove_unreferenced_vertices()
        trimesh.repair.fix_inversion(mesh)
        trimesh.repair.fix_normals(mesh)

        if len(mesh.vertices) < 3 or len(mesh.faces) < 1:
            logger.error("Mesh became invalid after repair.")
            return None

        # Normalize mesh
        center = mesh.bounds.mean(axis=0)
        mesh.apply_translation(-center)
        max_extent = np.max(np.linalg.norm(mesh.vertices, axis=1))
        if max_extent < 1e-6:
            logger.error("Mesh has zero or near-zero extent.")
            return None
        mesh.apply_scale(1.0 / max_extent)

        # Sample points
        result = sample_uniform_and_sharp_points(mesh, num_uniform_points=num_points//2, num_sharp_points=num_points//2)
        if result is None:
            logger.error("Point sampling returned None.")
            return None
        (uniform_points, uniform_normals), (sharp_points, sharp_normals) = result

        # Validate point clouds
        if uniform_points.shape[0] != num_points//2 or sharp_points.shape[0] != num_points//2:
            logger.error(f"Invalid point cloud sizes: uniform={uniform_points.shape[0]}, sharp={sharp_points.shape[0]}")
            return None

        surface_cloud = np.hstack([uniform_points, uniform_normals])
        sharp_cloud = np.hstack([sharp_points, sharp_normals])

        output = {
            "surface": torch.tensor(surface_cloud, dtype=dtype).unsqueeze(0),
            "sharp_surface": torch.tensor(sharp_cloud, dtype=dtype).unsqueeze(0)
        }
        logger.debug(f"Generated point clouds: surface shape={output['surface'].shape}, sharp_surface shape={output['sharp_surface'].shape}")
        return output
    except Exception as e:
        logger.error(f"Error processing mesh to VAE input: {str(e)}\n{traceback.format_exc()}")
        return None


In [ ]:
def process_animated_mesh_to_latents(glb_path, num_frames=16):
    logger.info(f"Processing GLB: {glb_path}")
    
    # Use the new, robust frame extraction function
    frame_meshes = extract_and_combine_frame_meshes(glb_path, num_frames)
    
    if frame_meshes is None:
        logger.error(f"Failed to extract frame meshes for {glb_path}")
        return None

    latents = []
    for i, frame_mesh in enumerate(frame_meshes):
        logger.debug(f"Processing frame {i+1}/{num_frames}")
        mesh_inputs = process_mesh_to_vae_input(frame_mesh)
        if mesh_inputs is None:
            logger.error(f"Failed to process mesh to VAE input for frame {i+1} in {glb_path}")
            return None

        mesh_inputs_on_device = {k: v.to(device) for k, v in mesh_inputs.items()}

        with torch.no_grad():
            _shape_embeds, kl_embed, _posterior = vae.encode(sample_posterior=True, **mesh_inputs_on_device)
            latent = kl_embed.squeeze(0).cpu()
        latents.append(latent)
        logger.debug(f"Successfully encoded frame {i+1} for {glb_path}")

    if len(latents) != num_frames:
        logger.error(f"Expected {num_frames} latents, got {len(latents)} for {glb_path}")
        return None

    sequence_latent = torch.stack(latents, dim=0) # Use torch.stack
    static_latent = latents[0]
    logger.info(f"Successfully processed {glb_path} with sequence shape {sequence_latent.shape}")
    return {"sequence": sequence_latent, "static": static_latent}


In [ ]:
def create_dataset_from_local_files(glb_paths, texts, num_frames=4):
    """
    Main function to process all animated meshes and texts from the local dataset.
    """
    logger.info("Starting dataset creation from local files...")
    processed_data = []

    for glb_path, caption in tqdm(zip(glb_paths, texts), total=len(glb_paths), desc="Processing Animated GLBs"):
        item = process_animated_mesh_to_latents(glb_path, num_frames)
        if item is None:
            print(f"\n--- !!! FAILED TO PROCESS: {glb_path} !!! ---")
            print("--- Check the log messages above this line for the specific reason. ---\n")
            continue

        # Tokenize text
        tokens = tokenizer(caption, padding="max_length", max_length=128, truncation=True, return_tensors="pt")

        processed_data.append({
            "x": item["sequence"],
            "y": {
                "input_ids": tokens["input_ids"].squeeze(0),
                "attention_mask": tokens["attention_mask"].squeeze(0),
            },
            "static_latent": item["static"]
        })

    logger.info(f"Successfully created dataset with {len(processed_data)} samples.")
    if not processed_data:
        logger.critical("Dataset is empty. Check logs for processing errors.")

    return processed_data

In [ ]:
class AnimDataset(Dataset):
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]


class CustomDataCollator:
    def __call__(self, features):
        batch = {}
        batch['x'] = torch.stack([f['x'] for f in features])
        y_features = [f['y'] for f in features]
        batch['y'] = {key: torch.stack([d[key] for d in y_features]) for key in y_features[0]}
        batch['static_latent'] = torch.stack([f['static_latent'] for f in features])
        return batch


class AnimDiTTrainer(Trainer):
    def __init__(self, *args, diffusion, **kwargs):
        super().__init__(*args, **kwargs)
        self.diffusion = diffusion

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        x_start = inputs.get("x")
        b, f, t, d = x_start.shape
        x_start = x_start.view(b, f * t, d)
        model_kwargs = {"y": inputs.get("y"), "static_latent": inputs.get("static_latent")}
        t = torch.randint(0, self.diffusion.num_timesteps, (x_start.shape[0],), device=x_start.device).long()
        noise = torch.randn_like(x_start)
        loss_dict = self.diffusion.training_losses(model, x_start, t, model_kwargs, noise=noise)
        loss = loss_dict["loss"].mean()
        return (loss, loss_dict) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        inputs = self._prepare_inputs(inputs)
        with torch.no_grad():
            loss = self.compute_loss(model, inputs)
        return (loss.detach(), None, None)



In [ ]:
def train_model():
    DATASET_PATH = "/kaggle/working/AI-Game-Engine/animated obj"
    if not os.path.isdir(DATASET_PATH):
        print("="*80)
        print(f"FATAL ERROR: Dataset directory not found at: '{DATASET_PATH}'")
        print("Please update the 'DATASET_PATH' variable in the 'train_model' function")
        print("to the correct location of your animated dataset folder.")
        print("="*80)
        return
        
    glb_paths, texts = load_local_dataset(dataset_dir=DATASET_PATH)
    all_data = create_dataset_from_local_files(glb_paths, texts)

    if not all_data:
        print("FATAL: Dataset creation failed. No data to train on. Exiting.")
        return

    random.shuffle(all_data)
    eval_size = max(1, int(len(all_data) * 0.10))
    train_data = all_data[eval_size:]
    eval_data = all_data[:eval_size]
    print(f"Data split: {len(train_data)} training samples, {len(eval_data)} evaluation samples.")

    train_dataset = AnimDataset(train_data)
    eval_dataset = AnimDataset(eval_data)
    data_collator = CustomDataCollator()

    diffusion = GaussianDiffusion(
        betas=get_named_beta_schedule("linear", 1000),
        model_mean_type=ModelMeanType.EPSILON,
        model_var_type=ModelVarType.FIXED_SMALL,
        loss_type=LossType.MSE,
    )

    # =================================================================
    #                       PHASE 1 TRAINING
    # =================================================================
    print("\n" + "="*50)
    print("      Starting AnimDiT Training: PHASE 1")
    print("="*50 + "\n")
    model_p1 = AnimDiT_models['AnimDiT-S'](
        input_tokens=2048,
        static_latent_tokens=2048,
        vocab_size=tokenizer.vocab_size,
        use_rope=False,
        use_ternary_rope=False,
        static_condition=True,
        num_frames=10,
        optimized_bitlinear=False,
        full_precision=False,
    ).to(device, dtype=dtype)

    training_args_p1 = TrainingArguments(
        output_dir="./animation_dit_phase1_checkpoint",
        num_train_epochs=200,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        learning_rate=1e-4,
        lr_scheduler_type="cosine",
        weight_decay=0.01,
        warmup_steps=200,
        logging_dir='./logs_phase1',
        logging_strategy="epoch",
        evaluation_strategy="epoch", # Changed from eval_strategy to evaluation_strategy for consistency
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True if device.type == "cuda" else False,
        max_grad_norm=1.0,
        dataloader_num_workers=2,
        remove_unused_columns=False,
        report_to=["wandb", "tensorboard"] if "WANDB_API_KEY" in os.environ else ["tensorboard"],
        run_name="AnimDiT-S-Phase1-Text-Static",
    )

    trainer_p1 = AnimDiTTrainer(
        model=model_p1,
        args=training_args_p1,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        diffusion=diffusion,
    )

    trainer_p1.train()
    trainer_p1.save_model("./animation_dit_phase1_final")
    tokenizer.save_pretrained("./animation_dit_phase1_final")
    print("Phase 1 training complete. Model saved to ./animation_dit_phase1_final")

    print("\nCleaning up old checkpoints to free up disk space...")
    shutil.rmtree('./animation_dit_phase1_checkpoint', ignore_errors=True)
    shutil.rmtree('./logs_phase1', ignore_errors=True)
    print("Cleanup complete.")

    # =================================================================
    #                       PHASE 2 TRAINING
    # =================================================================
    print("\n" + "="*50)
    print("      Starting AnimDiT Training: PHASE 2 (Fine-tuning)")
    print("="*50 + "\n")

    # Re-initialize model architecture for consistency, then load weights
    model_p2 = AnimDiT_models['AnimDiT-S'](
        input_tokens=2048,
        static_latent_tokens=2048, 
        vocab_size=tokenizer.vocab_size,
        use_rope=False,
        use_ternary_rope=False,
        static_condition=True,
        num_frames=10,
        full_precision = False,
        optimized_bitlinear=True,
    ).to(device, dtype=dtype)

    print("Loading weights from Phase 1 model for fine-tuning...")
    state_dict_p1 = safetensors_load("./animation_dit_phase1_final/model.safetensors", map_location="cpu")
    model_p2.load_state_dict(state_dict_p1)
    print("Weights loaded successfully.")

    training_args_p2 = TrainingArguments(
        output_dir="./animation_dit_checkpoint",  # Final checkpoint directory
        num_train_epochs=200,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        learning_rate=5e-5,
        lr_scheduler_type="cosine",
        weight_decay=0.01,
        warmup_steps=200,
        logging_dir='./logs_phase2',
        logging_strategy="epoch",
        evaluation_strategy="epoch", # Changed from eval_strategy to evaluation_strategy
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True if device.type == "cuda" else False,
        max_grad_norm=1.0,
        dataloader_num_workers=2,
        remove_unused_columns=False,
        report_to=["wandb", "tensorboard"] if "WANDB_API_KEY" in os.environ else ["tensorboard"],
        run_name="AnimDiT-S-Phase2-Text-Static",
    )

    trainer_p2 = AnimDiTTrainer(
        model=model_p2,
        args=training_args_p2,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        diffusion=diffusion,
    )

    trainer_p2.train()

    print("\nPhase 2 training complete. Saving the final model.")
    trainer_p2.save_model("./animation_dit_final")
    tokenizer.save_pretrained("./animation_dit_final")
    print("Final model saved to ./animation_dit_final")


if __name__ == "__main__":
    train_model()